In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

## Hyperparameters

In [10]:
BATCH_SIZE = 200

N_LAYERS = 1

LEARNING_RATES = 0.05

torch.manual_seed(689)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset preparations

In [3]:
from automata import *
from datas import even, sink, fix, unique

Generate dataset 'even'...
Done! in 3.9706 sec
Generate dataset 'sink'...
Done! in 34.9808 sec
Generate 'fix'...
Done! in 3.7940 sec
Generate 'unique'...
Done! in 3.5210 sec


In [4]:
even_torch = TorchData(even)

evenloader = DataLoader(even_torch, BATCH_SIZE, shuffle=True)

In [7]:
class automataRNN(nn.Module):

    def __init__(self, automaton:DFA) -> None:
        super().__init__()
        self.hidden_size = automaton.transition.shape[0]*automaton.transition.shape[1]

        self.rnn = nn.RNN(automaton.transition.shape[1], self.hidden_size, N_LAYERS, batch_first=True, device=device)
        self.toclass = nn.Linear(HIDDEN_SIZE, len(all_cat), device=device)

    def forward(self, x, truelen):
        "`truelen` is a list of the real length of the sequence : that way we can recover the good prediction along the gru"
        h0 = torch.zeros(N_LAYERS, x.shape[0], HIDDEN_SIZE).to(device)
        out, _ = self.gru(x.to(device), h0)
        out = torch.stack([out[i, truelen[i] -1, :] for i in range(out.shape[0])]) #extract only the require prediction y for each batch
        return self.toclass(out)
        
    def predict(self, x, truelen):
        return torch.argmax(self(x, truelen), dim = 1)
    
    def strpredict(self, name:str):
        tensorname = line_to_tensor_size(name, len(name))[None,:,:] # put a batch size of 0
        prediction = self.predict(tensorname, [len(name)])
        return all_categories[prediction.item()]


torch.Size([200, 22, 2])
